# **Chapter 9 – Up and running with TensorFlow**
_This notebook contains all the sample code and solutions to the exercises in chapter 9._

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# Creating and running a graph

In [2]:
import tensorflow as tf

tf.reset_default_graph()#用于清除默认图形堆栈，重置全局默认图。

x=tf.Variable(3,name="x")
y=tf.Variable(4,name="y")
f=x*x*y+y+2
f
#add冒号后面的数字编号表示这个张量是计算节点上的第几个结果

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [16]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)

42


In [6]:
sess.close()

In [8]:
#在with块，会有一个默认块
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()
result

42

In [12]:
#调用初始化器进行初始化
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()           #此时初始化
    result=f.eval()
result

42

In [13]:
#InteractiveSession 在创建时会将自己设置为默认会话
sess=tf.InteractiveSession()
init.run()
result=f.eval()
print(result)
sess.close()

42


# Managing graphs

In [24]:
x1=tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [20]:
#创建一个新图，用with块临时设为默认图
graph=tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
x2.graph is graph

True

In [23]:
x2.graph is tf.get_default_graph()

False

In [25]:
#节点值得生命周期
w=tf.constant(3)
x=w+2
y=x+5
z=x*3

with tf.Session() as sess:#运行两次
    print(y.eval())
    print(z.eval())

10
15


In [27]:
with tf.Session() as sess:
    y_val,z_val=sess.run([y,z])#只运行一次
    print(y_val)
    print(z_val)

10
15


# Linear Regression
### Using the Normal Equation

### 对数据进行分析

In [3]:
#  立刻下载数据集
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing(data_home="datasets/mldata", download_if_missing=True)

In [18]:
print(housing.data.shape)
print(type(housing.data))
print(housing.target.shape)
print(housing.feature_names)
print(housing.DESCR)

(20640, 8)
<class 'numpy.ndarray'>
(20640,)
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived

_read_csv()把第一行的数据变成了列名,这个csv的第一行并不是我们想象中的那样是一个列名。那样，我们处理数据的时候，就会出现问题，第一个不一致了嘛。
解决方法是设置参数！例如：df = pd.read_csv('1.csv', header=None, Names=['test']) _

In [6]:
#直接读取t下载好的文件
HOUSING_PATH = os.path.join("datasets", "mldata")
import pandas as pd
def load_housing_data(housing_path=HOUSING_PATH):
    path = os.path.join(housing_path, "CaliforniaHousing")
    csv_path = os.path.join(path, "cal_housing.data")
    return pd.read_csv(csv_path,header=None)           #header=None,
house = load_housing_data()              #都出来的数据，它把第一行的数据当成了索引，忽略掉了
house

,0,1,2,3,4,5,6,7,8
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0


In [15]:
#csv函数读取的数据
house_data = house.iloc[:, :-1]#[:-1]就是去除了的最后一列后剩下的部分。iloc根据位置索引
house_target = house.iloc[:, -1] #最后一列,返回的值有索引,如果只是获取一行数据的话，返回Serie对象
print(house_data.shape)
print(house_target.shape)
print(type(house_data))        #DataFrame相当于有表格，有行表头和列表头
print(type(house_target))      #Series 它是有索引，如果我们未指定索引，则是以数字自动生成，Series的俩个属性values和index获取内容和索引:

(20640, 8)
(20640,)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [8]:
# housing=fetch_california_housing得到的数据
print(housing.data.shape)
print(housing.target.shape)
print(housing.target)

(20640, 8)
(20640,)
[ 4.526  3.585  3.521 ...,  0.923  0.847  0.894]


In [27]:
print(housing.data[0,:])#不知道为什么housing.data和house中的数据有差距？？？？？
print(house_data.loc[0])    #Series对象
print(type(house_data.values))     #DataFrame转ndarray
print(house_data.values[0,:])

[   8.3252       41.            6.98412698    1.02380952  322.
    2.55555556   37.88       -122.23      ]
0   -122.2300
1     37.8800
2     41.0000
3    880.0000
4    129.0000
5    322.0000
6    126.0000
7      8.3252
Name: 0, dtype: float64
<class 'numpy.ndarray'>
[-122.23     37.88     41.      880.      129.      322.      126.
    8.3252]


# 使用house数据进行数据处理线性回归

In [28]:
m,n=house_data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), house_data]
housing_data_plus_bias[:,0]

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [30]:
#  Series的俩个属性values和index获取内容和索引:
#  Series对象（属于pandas库），不能直接用reshape函数(属于numpy库)， house_target.values.reshape(-1, 1)
print(house_target.shape)
print(house_target.index)
print(house_target.values)

(20640,)
RangeIndex(start=0, stop=20640, step=1)
[ 452600.  358500.  352100. ...,   92300.   84700.   89400.]


In [31]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")  
y = tf.constant(house_target.values.reshape(-1, 1), dtype=tf.float32, name="y")  #reshape(-1, 1)变成一列
XT = tf.transpose(X)  #转置
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)  

with tf.Session() as sess:  
    theta_value = theta.eval()  
print(theta_value) 

[[ -3.53207150e+06]
 [ -4.21040938e+04]
 [ -4.19390430e+04]
 [  1.16819812e+03]
 [ -8.33691406e+00]
 [  1.13105164e+02]
 [ -3.85289040e+01]
 [  4.95863571e+01]
 [  4.03551016e+04]]


# 使用housing数据进行线性回归

In [32]:
tf.reset_default_graph()#用于清除默认图形堆栈，重置全局默认图。
m, n = housing.data.shape
# 这里添加一个额外的bias输入特征(x0=1)到所有的训练数据上面，因为使用的numpy所有会立即执行
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
# 创建两个TensorFlow常量节点X和y，持有数据和标签
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)   #转置成列向量
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)  #使用normal equation的方法求解theta，之前线性模型中有提及

with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[ -3.64127045e+01],
       [  4.37925577e-01],
       [  9.52987000e-03],
       [ -1.08451903e-01],
       [  6.48505449e-01],
       [ -3.66319728e-06],
       [ -3.80029809e-03],
       [ -4.15675610e-01],
       [ -4.28362131e-01]], dtype=float32)

### _Compare with pure NumPy_

In [62]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


### _Compare with  Scikit-Learn_

In [63]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])#截距，斜率

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


# Using Batch Gradient Descent实现梯度下降
当使用梯度下降时，请记住，首先要对输入特征向量进行归一化，否则训练可能要慢得多

In [35]:
from sklearn.preprocessing import StandardScaler
## StandardScaler默认就做了方差归一化，和均值归一化，这两个归一化的目的都是为了更快的进行梯度下降
scaler=StandardScaler()
scaled_housing_data=scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias=np.c_[np.ones((m,1)),scaled_housing_data]
print(scaled_housing_data_plus_bias.shape)

(20640, 9)


### Manually computing the gradients手动计算

In [45]:
tf.reset_default_graph()
n_epochs=1000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
## random_uniform函数创建图里一个节点包含随机数值，给定它的形状和取值范围
theta=tf.Variable(tf.random_uniform([n+1,1],-1,1),name='theta')   #使用gradient需要有一个初值
y_pred=tf.matmul(X,theta,name="predictions")   #x是m*（n+1），theta是（n+1）*1
error=y_pred-y
#TensorFlow求解均值功能强大
mse=tf.reduce_mean(tf.square(error),name="mse")
# 梯度的公式：(y_pred - y) * xj
#自己写训练过程，实际可以采用TensorFlow自带的功能更强大的自动求解autodiff方法
gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()      #等价 sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        training_op.eval()  #等价 sess.run(training_op)
    best_theta=theta.eval()
best_theta

Epoch 0 MSE= 8.90509
Epoch 100 MSE= 0.704832
Epoch 200 MSE= 0.575422
Epoch 300 MSE= 0.561636
Epoch 400 MSE= 0.553213
Epoch 500 MSE= 0.54687
Epoch 600 MSE= 0.542014
Epoch 700 MSE= 0.538272
Epoch 800 MSE= 0.535375
Epoch 900 MSE= 0.53312


array([[ 2.06855226],
       [ 0.88492042],
       [ 0.14619108],
       [-0.33808011],
       [ 0.35164922],
       [ 0.00476077],
       [-0.04282646],
       [-0.64792496],
       [-0.62347209]], dtype=float32)

### Using autodiff自动微分
Same as above except for the gradients = ... line:

In [46]:
tf.reset_default_graph()
n_epochs=1000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
## random_uniform函数创建图里一个节点包含随机数值，给定它的形状和取值范围
theta=tf.Variable(tf.random_uniform([n+1,1],-1,1),name='theta')   #使用gradient需要有一个初值
y_pred=tf.matmul(X,theta,name="predictions")   #x是m*（n+1），theta是（n+1）*1
error=y_pred-y
#TensorFlow求解均值功能强大
mse=tf.reduce_mean(tf.square(error),name="mse")
# 梯度的公式：(y_pred - y) * x



#采用TensorFlow自带的功能更强大的自动求解autodiff方法
gradients = tf.gradients(mse, [theta])[0]



training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()      #等价 sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        training_op.eval()  #等价 sess.run(training_op)
    best_theta=theta.eval()
best_theta

Epoch 0 MSE= 7.75823
Epoch 100 MSE= 0.819722
Epoch 200 MSE= 0.670463
Epoch 300 MSE= 0.632047
Epoch 400 MSE= 0.604961
Epoch 500 MSE= 0.584951
Epoch 600 MSE= 0.570107
Epoch 700 MSE= 0.559058
Epoch 800 MSE= 0.550802
Epoch 900 MSE= 0.544609


array([[ 2.06855249],
       [ 0.87316912],
       [ 0.16255787],
       [-0.28114563],
       [ 0.28929538],
       [ 0.01086778],
       [-0.04396051],
       [-0.53965646],
       [-0.51215559]], dtype=float32)

### Using a `GradientDescentOptimizer`使用优化器

In [49]:
tf.reset_default_graph()
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#也可以使用更快的动量优化器
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("epoch",epoch,"MSE=",mse.eval())
        #training_op.eval()   #'Operation' object has no attribute 'eval'可能是因为它没有输出
        sess.run(training_op)
    best_theta=theta.eval()

epoch 0 MSE= 2.75443
epoch 100 MSE= nan
epoch 200 MSE= nan
epoch 300 MSE= nan
epoch 400 MSE= nan
epoch 500 MSE= nan
epoch 600 MSE= nan
epoch 700 MSE= nan
epoch 800 MSE= nan
epoch 900 MSE= nan


# Mini-batch Gradient Descent
为了实现小批量梯度下降（Mini-batch Gradient Descent）。我们需要一种在每次迭代时用下一个小批量替换X和Y的方法。 最简单的方法是使用占位符（placeholder）节点
### Placeholder nodes

In [69]:
tf.reset_default_graph()
n_epochs = 10  
learning_rate = 0.01
#批次的大小
batch_size=100
n_batches=int(np.ceil(m/batch_size))

#定义占位符
X=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="y")
theta=tf.Variable(tf.random_uniform((n+1,1),-1,1,seed=42),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="MSE")
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

def fetch_batch(epoch,batch_index,batch_size):
    np.random.seed(epoch*n_batches+batch_index)
    indices=np.random.randint(m,size=batch_size)#从0到m中，随机取出100个索引
    X_batch=scaled_housing_data_plus_bias[indices]
    y_batch=housing.target.reshape(-1,1)[indices]
    return X_batch,y_batch

In [70]:
init = tf.global_variables_initializer()
saver = tf.train.Saver({"weights": theta})   #在构造阶段结束（创建所有变量节点之后）创建一个保存节点

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    best_theta=theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")    #调用保存方法

In [53]:
best_theta

array([[ 2.07001591],
       [ 0.82045609],
       [ 0.11731728],
       [-0.22739054],
       [ 0.31134021],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643941],
       [-0.87950087]], dtype=float32)

In [56]:
#恢复模型：在构建阶段结束时创建一个保存器，就像之前一样，在执行阶段的开始，不用init节点初始化变量，调用restore()方法的保存器对象
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()        # 执行

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [57]:
np.allclose(best_theta, best_theta_restored)#结果完全相同

True

# Visualizing the graph

In [88]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [89]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [90]:
#放在构造期的最后
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [95]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()   

In [93]:
file_writer.close()

# Name scopes命名作用域

In [99]:
tf.reset_default_graph()
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)


learning_rate=0.01
n_epochs=10
batch_size=100
n_batches=int(np.ceil(m/batch_size))

X=tf.placeholder(dtype=tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(dtype=tf.float32,shape=(None,1),name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1,1),name="theta")
y_pred=tf.matmul(X,theta)

with tf.name_scope("loss") as scope:#命名作用域
    error=y_pred-y
    mse=tf.reduce_mean(tf.square(error),name="MSE")

optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)    
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()
mse_summary=tf.summary.scalar('MSE',mse)
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str=mse_summary.eval(feed_dict={X:X_batch,y:y_batch})
                step=epoch*n_batches
                file_writer.add_summary(summary_str,step)   #将训练过程数据保存在filewriter指定的文件中
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:",best_theta)

Best theta: [[ 2.06999016]
 [ 0.81566656]
 [ 0.11626159]
 [-0.21896598]
 [ 0.30451947]
 [ 0.00324846]
 [-0.01099628]
 [-0.92937011]
 [-0.89179307]]


In [98]:
print(error.op.name)

loss/sub
